### Case Study Question
---
    1. What is the total amount each customer spent at the restaurant?

In [1]:
SELECT
  s.customer_id as customer, 
  SUM(mn.price) as total_amount
FROM 
  sales s
JOIN menu mn on mn.product_id = s.product_id
GROUP BY s.customer_id

(3 rows affected)

Total execution time: 00:00:00.022

customer,total_amount
A,76
B,74
C,36


    2. How many days has each customer visited the restaurant?

In [2]:
SELECT
  customer_id, 
  COUNT(DISTINCT order_date) as num_days
FROM
  sales
GROUP BY customer_id;

(3 rows affected)

Total execution time: 00:00:00.014

customer_id,num_days
A,4
B,6
C,2


    3. What was the first item from the menu purchased by each customer?

In [3]:
WITH temp AS (
  SELECT
    s.customer_id,
    s.order_date,
    mn.product_name,
    DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY s.order_date ASC) AS dense_rank
  FROM
    sales s
  JOIN menu mn ON mn.product_id = s.product_id
)
SELECT customer_id as customer, order_date, product_name as first_item_purchased
FROM
  temp
WHERE
  dense_rank = 1
GROUP BY
  customer_id, order_date, product_name

(4 rows affected)

Total execution time: 00:00:00.023

customer,order_date,first_item_purchased
A,2021-01-01,curry
A,2021-01-01,sushi
B,2021-01-01,curry
C,2021-01-01,ramen


    4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [4]:
SELECT TOP 1
  mn.product_name, COUNT(mn.price) as frequency, SUM(mn.price) as purchased
FROM sales s
JOIN menu mn on mn.product_id = s.product_id
GROUP BY mn.product_name
ORDER BY purchased DESC

(1 row affected)

Total execution time: 00:00:00.033

product_name,frequency,purchased
ramen,8,96


    5. Which item was the most popular for each customer?

In [5]:
WITH tbl1 as(
  SELECT
    s.customer_id, mn.product_name, COUNT(mn.product_id) as frequency
  FROM
    sales s
  JOIN menu mn on mn.product_id = s.product_id
  GROUP BY s.customer_id, mn.product_name
),
tbl2 as(
  SELECT
    tbl1.customer_id,
    tbl1.product_name,
    DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY frequency DESC) AS dense_rank
  FROM tbl1
)
SELECT
  tbl2.customer_id,
  tbl2.product_name
FROM
  tbl2
WHERE
  dense_rank = 1

(5 rows affected)

Total execution time: 00:00:00.023

customer_id,product_name
A,ramen
B,sushi
B,curry
B,ramen
C,ramen


    6. Which item was purchased first by the customer after they became a member?

In [6]:
WITH tbl1 as(
  SELECT
    s.customer_id,
    s.order_date,
    mn.product_name
  FROM sales s
  JOIN members mb on mb.customer_id = s.customer_id
  JOIN menu mn on mn.product_id = s.product_id  
  WHERE s.order_date >= mb.join_date
),
 tbl2 as(
  SELECT
    tbl1.customer_id,
    tbl1.order_date,
    tbl1.product_name,
    DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY order_date ASC) as dense_rank
  FROM
    tbl1
 )
 SELECT
  tbl2.customer_id,
  tbl2.order_date,
  tbl2.product_name
FROM tbl2
WHERE
  dense_rank = 1

(2 rows affected)

Total execution time: 00:00:00.020

customer_id,order_date,product_name
A,2021-01-07,curry
B,2021-01-11,sushi


    7. Which item was purchased just before the customer became a member?

In [7]:
WITH tbl1 as(
  SELECT
    s.customer_id,
    s.order_date,
    mn.product_name
  FROM sales s
  JOIN members mb on mb.customer_id = s.customer_id
  JOIN menu mn on mn.product_id = s.product_id  
  WHERE s.order_date < mb.join_date
),
tbl2 as(
  SELECT
    tbl1.customer_id,
    tbl1.order_date,
    tbl1.product_name,
    DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY order_date DESC) as dense_rank
  FROM
    tbl1
)
SELECT
  tbl2.customer_id,
  tbl2.order_date,
  tbl2.product_name
FROM tbl2
WHERE
  dense_rank = 1

(3 rows affected)

Total execution time: 00:00:00.019

customer_id,order_date,product_name
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-04,sushi


    8. What is the total items and amount spent for each member before they became a member?

In [8]:
SELECT
  s.customer_id,
  COUNT(s.product_id) as total_item,
  SUM(mn.price) as total_amount_spent
FROM sales s
JOIN menu mn on mn.product_id = s.product_id
JOIN members mb on mb.customer_id = s.customer_id
WHERE s.order_date < mb.join_date
GROUP BY s.customer_id;

(2 rows affected)

Total execution time: 00:00:00.013

customer_id,total_item,total_amount_spent
A,2,25
B,3,40


    9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [9]:
WITH tbl1 as(
  SELECT
    s.customer_id,
    s.product_id,
    COUNT(s.product_id) as quantity,
    mn.price
  FROM sales s
  JOIN menu mn on mn.product_id = s.product_id
  GROUP BY s.customer_id, s.product_id, mn.price
),
tbl2 as(
  SELECT
    customer_id,
    CASE
      WHEN product_id = 1 THEN price*quantity*20
      ELSE price*quantity*10
    END as point
  FROM tbl1
)
SELECT
  customer_id,
  SUM(point) as points
FROM tbl2
GROUP BY customer_id;

(3 rows affected)

Total execution time: 00:00:00.015

customer_id,points
A,860
B,940
C,360


10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [10]:
WITH tbl1 as(
  SELECT
    s.customer_id,
    s.order_date,
    mb.join_date,
    s.product_id,
    COUNT(s.product_id) as quantity,
    mn.price
  FROM sales s
  JOIN members mb on mb.customer_id = s.customer_id
  JOIN menu mn on mn.product_id = s.product_id
  WHERE MONTH(s.order_date) = 1
  GROUP BY s.customer_id, s.order_date, mb.join_date, s.product_id, mn.price
),
tbl2 as(
  SELECT
    customer_id,
    product_id,
    CASE
      WHEN order_date <= DATEADD(DAY, 6 ,join_date) and order_date >= join_date THEN price*quantity*20
      ELSE
        CASE
          WHEN product_id = 1 THEN price*quantity*20
          ELSE price*quantity*10
        END
    END as point
  FROM tbl1
)
SELECT
  customer_id,
  SUM(point) as point
FROM tbl2
GROUP BY customer_id;

(2 rows affected)

Total execution time: 00:00:00.014

customer_id,point
A,1370
B,820
